# Import packages and set the device

In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

torch.manual_seed(1015)

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

# Load data

In [2]:
train = pd.read_csv('../data/train.csv', encoding='euc-kr')
train['DateTime'] = pd.to_datetime(train.DateTime)
train['date'] = train.DateTime.dt.date
train = train.groupby('date').sum().reset_index()
train

,date,사용자,세션,신규방문자,페이지뷰
0,2018-09-09,281,266,73,1826
1,2018-09-10,264,247,51,2092
2,2018-09-11,329,310,58,1998
3,2018-09-12,300,287,45,2595
4,2018-09-13,378,344,50,3845
...,...,...,...,...,...
787,2020-11-04,4516,4472,1196,112683
788,2020-11-05,4155,4037,1044,102901
789,2020-11-06,3663,3576,825,88015
790,2020-11-07,2472,2417,531,57386


# Preprocess data

In [5]:
# Min-max scaling
mini = train.iloc[:, 1:].min()
size = train.iloc[:, 1:].max() - train.iloc[:, 1:].min()
train.iloc[:, 1:] = (train.iloc[:, 1:] - mini) / size

input_window = 30
output_window = 7

window_x = np.zeros((train.shape[0] - (input_window + output_window), input_window, 4))
window_y = np.zeros((train.shape[0] - (input_window + output_window), output_window, 4))

for start in range(train.shape[0] - (input_window + output_window)):
    end = start + input_window
    window_x[start, :, :] = train.iloc[start : end                , 1:].values
    window_y[start, :, :] = train.iloc[end   : end + output_window, 1:].values

# Define a model

In [12]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            batch_first=True)
        self.hidden_lstm = nn.LSTM(input_size=hidden_size,
                                   hidden_size=hidden_size,
                                   batch_first=True)
        self.time_fc = nn.Linear(hidden_size, 4)
    
    def forward(self, x_time):
        out_time, _ = self.lstm(x_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time = self.time_fc(out_time[:, -7:, :])
        return out_time.view(-1, 7, 4)
    
model = LSTM(input_size=4, hidden_size=30).to(device)

# Training

In [79]:
window_x = torch.tensor(window_x).float().to(device)
window_y = torch.tensor(window_y).float().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.MSELoss(size_average=True)
num_epochs = 500
train_error = []

for t in range(num_epochs):
    train_pred = model(window_x)
    loss = criterion(train_pred, window_y)
    train_error.append(loss)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if t % 10 == 0:
        print(f"{t} epochs. train MSE: {loss.item():1.5f}")

10 Epochs train MSE: 0.02835
20 Epochs train MSE: 0.01247
30 Epochs train MSE: 0.01252
40 Epochs train MSE: 0.00992
50 Epochs train MSE: 0.00823
60 Epochs train MSE: 0.00807
70 Epochs train MSE: 0.00796
80 Epochs train MSE: 0.00788
90 Epochs train MSE: 0.00779
100 Epochs train MSE: 0.00768
110 Epochs train MSE: 0.00748
120 Epochs train MSE: 0.00708
130 Epochs train MSE: 0.00704
140 Epochs train MSE: 0.00642
150 Epochs train MSE: 0.00630
160 Epochs train MSE: 0.00614
170 Epochs train MSE: 0.00605
180 Epochs train MSE: 0.00592
190 Epochs train MSE: 0.00568
200 Epochs train MSE: 0.00554
210 Epochs train MSE: 0.00578
220 Epochs train MSE: 0.00613
230 Epochs train MSE: 0.00559
240 Epochs train MSE: 0.00613
250 Epochs train MSE: 0.00543
260 Epochs train MSE: 0.00507
270 Epochs train MSE: 0.00485
280 Epochs train MSE: 0.00459
290 Epochs train MSE: 0.00442
300 Epochs train MSE: 0.00426
310 Epochs train MSE: 0.00419
320 Epochs train MSE: 0.00379
330 Epochs train MSE: 0.00362
340 Epochs train MS

# Prediction

In [80]:
submission = pd.read_csv('../data/submission.csv', encoding='euc-kr')

last_month = train.iloc[-30:,1:].values[np.newaxis,...]
last_month = torch.tensor(last_month).float().to(device)

for start in range((len(submission) - output_window) // 7 + 2):
    start = start * 7
    next_week = model(last_month)
    last_month = torch.cat([last_month[:,7:,:], next_week], axis = 1)

    pred_week = next_week.cpu().detach().numpy().reshape(output_window, 4)
    pred_week = pred_week * size.values + mini.values
    pred_week = pred_week.astype(int)

    if start / 7 == (len(submission) - output_window) // 7 + 1:
        submission.iloc[start:, 1:] = pred_week[-submission.iloc[start:, 1:].shape[0]:, :]
    else:
        submission.iloc[start:start+output_window, 1:] = pred_week

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2020-11-09,2102,2101,349,52157
1,2020-11-10,4940,4955,1273,128364
2,2020-11-11,5612,5614,1552,149229
3,2020-11-12,4441,4490,1199,120150
4,2020-11-13,3952,4140,1122,107881
...,...,...,...,...,...
56,2021-01-04,2177,2163,523,47459
57,2021-01-05,2165,2144,509,47366
58,2021-01-06,1589,1564,317,32201
59,2021-01-07,1501,1439,260,31169


In [ ]:
submission

# Save submission file

In [81]:
submission.to_csv('submission.csv', index=False, encoding='euc-kr')